## DERIVATIVES PRICER 
* Code is centered around tree-pricing models and monte carlo simulations

### TO DO: 
* Find a proper real-life data source to create robust examples
    * Potential Sources:
        * https://finance.yahoo.com/quote/%5ESPX/options?guce_referrer=aHR0cDovL2xvY2FsaG9zdDo4ODg4Lw&guce_referrer_sig=AQAAAJUeFD2zHulQJDrOgwLtyv-FW-lil1QsNe3h00p50uKsA4L4mKOpmN9jBZXdlzRIx2B-NivxYckMXQSwP3T3DzQyM0PdQ1t1hfrYDqgFKRRAuvBUOlS-JFjdfxwWN0U9_BHvujq_PlghcFhdQhcvnLrDonM3ti4yhPu1MD8jBksd&p=%5ESPX&straddle=false
        * https://www2.asx.com.au/markets/trade-our-derivatives-market/derivatives-market-prices/single-stock-derivatives?expiryMonths=Feb+2021&expiryMonths=Mar+2021&expiryMonths=Apr+2021&expiryMonths=May+2021&expiryMonths=Jun+2021&expiryMonths=Jul+2021&expiryMonths=Aug+2021&expiryMonths=Sep+2021&expiryMonths=Oct+2021&expiryMonths=Nov+2021&expiryMonths=Dec+2021&expiryMonths=Jan+2022&expiryMonths=Feb+2022&expiryMonths=Mar+2022&expiryMonths=Apr+2022&expiryMonths=May+2022&expiryMonths=Jun+2022&expiryMonths=Jul+2022&expiryMonths=Aug+2022&expiryMonths=Sep+2022&expiryMonths=Oct+2022&expiryMonths=Nov+2022&expiryMonths=Dec+2022&expiryMonths=Jan+2023&expiryMonths=Feb+2023&expiryMonths=Mar+2023&expiryMonths=Apr+2023&expiryMonths=May+2023&expiryMonths=Jun+2023&expiryMonths=Jul+2023&expiryMonths=Aug+2023&expiryMonths=Sep+2023&expiryMonths=Oct+2023&expiryMonths=Nov+2023&expiryMonths=Dec+2023&expiryMonths=Jan+2024&expiryMonths=Feb+2024&expiryMonths=Mar+2024&expiryMonths=Apr+2024&expiryMonths=May+2024&expiryMonths=Jun+2024&expiryMonths=Jul+2024&expiryMonths=Aug+2024&expiryMonths=Sep+2024&expiryMonths=Oct+2024&expiryMonths=Nov+2024&expiryMonths=Dec+2024&expiryMonths=Jan+2025
        
        
* Add vectorised MC 
* Finish tree methods & add example
* Add variance reduction methods to MC
* Show how non-closed solution american options can be priced from the base of the asian MC sim
* Add visuals for demonstrating the impact of IV on pricing
* Add GARCH models for volatility forecasting
* Price and backtest real traded options to see the accuracy of said models in practice (ex. liquidity constraints)

In [1]:
## import packages
from scipy.stats import norm
from scipy.optimize import brentq
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import pandas_datareader.data as web
%matplotlib inline

## BLACKSCHOLES - BASELINE
* (CLOSED FORM SOLUTION FOR EUROPEAN OPTIONS)

In [38]:
# hypothetical parameters:
S0=11.; K=10.; T=3/12.; r=.02; sigma=.3; N=500;

In [39]:

def blackscholes(S0, K, T, r, sigma):
    """
    Price of a European call in the Black-Scholes model.
    """
    d1 = (np.log(S0)-np.log(K)+(r+sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1-sigma*np.sqrt(T)
    return S0*norm.cdf(d1)-np.exp(-r*T)*K*norm.cdf(d2)

In [40]:
blackscholes(S0, K, T, r, sigma)

1.2858368491569285

In [41]:
# also pass in an ndarray for for different strikes)
Ks = np.linspace(8, 10, 5)
blackscholes(S0, Ks, T, r, sigma)

array([3.04764278, 2.56561793, 2.10292676, 1.67202011, 1.28583685])

## IMPLIED VOLATILITY

* The *implied volatility* (IV, $\sigma _{I}$) of an option is that value of $\sigma $ which equates the BS model price to the observed market price $C_{0}^{obs}$, i.e., it solves
<br><br>
$$
C_{0}^{obs}=BS(S_{0},K,T,r,\sigma_I).
$$
<br>
* If the BS assumptions were correct, then any option traded on the asset should have the same IV, which should in turn equal historical volatility.

* In practice, options with different strikes $K$ and hence *moneyness* $K/S_{0}$ have different IVs: this gives rise to a *volatility smile* or *smirk/skew*.

    * Also, options with different times to maturity have different IVs: this yields a *volatility term structure*.

* These phenomena are evidence of a failure of the assumptions of the Black-Scholes model, most importantly that of a constant volatility $\sigma$.

In [45]:
def impvol(S0, K, T, r, C_obs, Type='call'): 
    """
    Implied Black-Scholes volatility. 
    Root Finding Function: Scipy Brentq
    """
    
    ## if != call; can convert to call price via parity
    if Type == 'put':  
        C_obs = C_obs+S0-np.exp(-r*T)*K
        
    ## else if type = call (function default behaviour)    
    # theranchi bounds
    L = -2*norm.ppf((S0-C_obs)/(2.0*min(S0, np.exp(-r*T)*K)))/np.sqrt(T)
    U = -2*norm.ppf((S0-C_obs)/(S0+np.exp(-r*T)*K))/np.sqrt(T)
    
    #Partial application would be:  f(s)=BS(S0, K, T, r, s)-C_obs.
    return brentq(lambda s: blackscholes(S0, K, T, r, s)-C_obs, L, U)  

In [46]:
## dummy data for now 
C_obs=3 

# hypothetical parameters:
S0=11.; K=10.; T=3/12.; r=.02; sigma=.3; N=500;

In [47]:
IV = impvol(S0, K, T, r, C_obs); (IV, blackscholes(S0, K, T, r, IV))

(1.1854766702441244, 3.0000000000000178)

In [49]:
# equivalent to re-running the blackscholes from above with the implied volatility (IV) in place of sigma
blackscholes(S0, K, T, r, IV) # IV replaces sigma

3.0000000000000178

## TREE OPTION PRICING 
* Specifications: 
    * Call / Put
    * American / European

In [15]:
def optiontree_flexible(S0, K, T, r, sigma, N, Type='call', option_spec='american'):
    """
    Calculate (American or European) (call & put) prices based on a N-step binomial tree: 
    """
    
    # define delta_t (Time to maturity / number of steps)
    deltaT = T / float(N)
    
    # define the factor by which the (up) movement rises
    u = np.exp(sigma*np.sqrt(deltaT))
    
    # define d in such a way the tree is recombinant 
    d = 1.0 / u
    
    # define probabilities according to the theory of risk neutral valuation
    p = (np.exp(r*deltaT)-d) / (u-d)
    
    # probabilities (up) and (down) = piu and pid respectively
    piu = np.exp(-r*deltaT) * p
    pid = np.exp(-r*deltaT) * (1-p)
    
    # calculate the stock price tree (stock price at origination * up(down) movement factors relative to position in tree
    S = S0 * u**np.arange(N+1) * d**(2*np.arange(N+1)[:, np.newaxis])
    
    #keep only the upper triangular part (as only non-negative payoffs contribute to price) 
    S = np.triu(S)  
    
    if Type =='call':
        
        # initialise the matrix of zeros to be written over for (when payoff > 0) for the payoff structure 
        C = np.zeros((N+1, N+1))
        
        # calculate the payoff for each node in the tree through to (N) steps: larger of (0, or stock - strike price) for call
        C[:, N] = np.maximum(0, S[:, N]-K) 

        # iterate backwards through tree to multiply payoffs by probability of being in that node
        for j in range(N-1, -1, -1):
            C[:j+1, j] = piu * C[:j+1, j+1] + pid * C[1:j+2, j+1]
            
            # if option = American; calculate the value of early exercise
            if option_spec == 'american':
                C[:j+1, j] = np.maximum(K-S[:j+1, j], C[:j+1, j] )

        # return the price of the option at origination (which is the weighted sum of payoffs * probabilities) in the tree    
        return  np.round(C[0, 0],6)
    
    elif Type =='put':
        # initialise matrix of zeros (could use C again but for clarity, rename it to P)
        P = np.zeros((N+1,N+1))
        
        # calculate payoff- for put is: maximum of (0, strike-stock)
        P[:, N]=np.maximum(0, K-S[:, N])
        
        # iterate through tree of payoffs and probabilities
        for j in range(N-1, -1, -1):
            P[:j+1, j] = piu * P[:j+1, j+1] + pid * P[1:j+2, j+1]
            
            # if option = American; calculate the value of early exercise
            if option_spec =='american':
                P[:j+1, j] = np.maximum(P[:j+1, j], K-S[:j+1, j] )
    
        # return price at origination
        return  np.round(P[0, 0],6)    
    
    

In [4]:
# hypothetical parameters:
S0=11.; K=10.; T=3/12.; r=.02; sigma=.3; N=500;

In [11]:
optiontree_flexible(S0, K, T, r, sigma, N, Type='call', option_spec='european')

1.28574

In [12]:
optiontree_flexible(S0, K, T, r, sigma, N, Type='put', option_spec='european')

0.235864

In [13]:
optiontree_flexible(S0, K, T, r, sigma, N, Type='call', option_spec='american')

1.522124

In [34]:
optiontree_flexible(S0, K, T, r, sigma, N, Type='put', option_spec='american')

0.236918

In [35]:
def impvol_tree_model(S0, K, T, r, price_obs, N, option_type='put'):
    """
    Tree-implied volatility for various option types
    """
    
    # Theranchi Bounds (L, U)
    # These bounds are used to reprove asymptotic formulae for implied volatility at extreme strikes and/or maturities.
    L = 0.1
    U = 10
    
    if option_type =='put':
        return brentq(lambda s: optiontree_flexible(S0, K, T, r, s, N)-price_obs, L, U)
    
    # else "return call"
    else:
        return brentq(lambda s: (price_obs, L, U) - optiontree_flexible(S0, K, T, r, s, N))
    
    

In [36]:
# hypothetical obs_price 
price_obs = 0.3

# hypothetical parameters:
S0=11.; K=10.; T=3/12.; r=.02; sigma=.3; N=500;

In [37]:
impvol_tree_model(S0, K, T, r, price_obs, N)

ValueError: f(a) and f(b) must have different signs

## INCOMPLETE !! 



# MONTE CARLO SIMULATIONS

## define stochastic path simulators 

In [8]:
def bmsim(T, N, X0=0, mu=0, sigma=1):
    """Simulate a Brownian motion path with X0/mu = 0 drift terms"""
    
    deltaT = float(T)/N
    
    #N+1 is one more than we need, actually. This way we won't have to grow dX by X0.
    tvec = np.linspace(0, T, N+1)
    z = np.random.randn(N+1)   
    dX = mu*deltaT + sigma*np.sqrt(deltaT)*z  #X[j+1]-X[j]=mu*deltaT + sigma*np.sqrt(deltaT)*z[j].
    dX[0] = 0.
    X = np.cumsum(dX)
    X += X0    
    return tvec, X

In [ ]:
def gbmsim(T, N, S0=1, mu=0, sigma=1):
    """Simulate a Geometric Brownian motion path."""
    deltaT = float(T)/N
    tvec = np.linspace(0, T, N+1)
    z = np.random.randn(N+1)  #Again one more than we need. This keeps it comparable to bmsim.
    S = np.zeros_like(z)
    S[0] = S0
    for i in range(0, N): #Note: we can no longer vectorize this, because S[:, j] is needed for S[:, j+1].
        S[i+1] = S[i] + mu*S[i]*deltaT + sigma*S[i]*np.sqrt(deltaT)*z[i+1]
    return tvec, S

## ARITHMETIC AVG ASIAN CALL 

In [9]:

def asianmc(S0, K, T, r, sigma, q, N, numsim=10000):
    """Monte Carlo price of an arithmetic average Asian call."""
    X0 = np.log(S0)
    nu = r-q-.5*sigma**2
    payoffs = np.zeros(numsim)
    for i in range(numsim):
        _, X = bmsim(T, N, X0, nu, sigma)  #Convention: underscore holds value to be discarded.
        S = np.exp(X)
        payoffs[i] = max(S[1:].mean()-K, 0.)  #We ignore S0 in the mean.
    g = np.exp(-r*T)*payoffs
    C = g.mean(); s = g.std()
    zq = norm.ppf(0.975)
    Cl = C-(zq*s)/np.sqrt(numsim)
    Cu = C+(zq*s)/np.sqrt(numsim)
    return C, Cl, Cu

In [10]:
%%time

S0 = 11; K = 10; T = 3/12.; r = 0.02; sigma = .3; q = 0.01; N = 10

# ensure random seed for comparability
np.random.seed(0)

C0, Cl, Cu = asianmc(S0, K, T, r, sigma, q, N); C0, Cl, Cu

Wall time: 511 ms


(1.0927262054551385, 1.0747653929130998, 1.1106870179971773)

In [ ]:
## vectorise the BM paths

In [5]:
#Note new input: numsim, the number of paths.
## check -- error in code when plugging into MC! 

def bmsim_vec(T, N, X0=0, mu=0, sigma=1, numsim=1):  
    """Simulate `numsim` Brownian motion paths."""
    deltaT = float(T)/N
    tvec = np.linspace(0, T, N+1)
    z = np.random.randn(numsim, N+1)  
    dX = mu*deltaT + sigma*np.sqrt(deltaT)*z
    
    dX[:, 0] = 0.  
    X = np.cumsum(dX, axis=1)  
    
    X += X0    
    return tvec, X

In [ ]:
def asianmc_vec(S0, K, T, r, sigma, q, N, numsim=10000):
    """Monte Carlo price of an arithmetic average Asian call. Vectorised Version"""
    X0 = np.log(S0)
    nu = r-q-.5*sigma**2    
    
    #simulate all paths at once:
    _, X = bmsim_vec(T, N, X0, nu, sigma, numsim)
    S = np.exp(X)
    payoffs = np.maximum(S[:, 1:].mean(axis=1)-K, 0.)  #S[1:]->S[:, 1:], max->maximum, mean()->mean(axis=1)
    g = np.exp(-r*T)*payoffs
    C = g.mean(); s = g.std()
    zq = norm.ppf(0.975)
    Cl = C - zq/np.sqrt(numsim)*s
    Cu = C + zq/np.sqrt(numsim)*s
    return C, Cl, Cu

## Antithetic Sampling (Variance Reduction Technique)

In [ ]:
def bmsim_vec(T, N, X0=0, mu=0, sigma=1, numsim=1, av=False):
    """Simulate `numsim` Brownian motion paths. If av=True, then 2*`numsim` paths are returned,
    where paths numsim:2*numsim+1 are the antithetic paths."""
    deltaT = float(T)/N
    tvec = np.linspace(0, T, N+1)
    z = np.random.randn(numsim, N+1)
    if av:
        z = np.concatenate((z, -z))  #yields an array of shape (2*numsim,N+1); z stacked on -z
    dX = mu*deltaT + sigma*np.sqrt(deltaT)*z
    dX[:, 0] = 0.
    X = np.cumsum(dX, axis=1)
    X += X0    
    return tvec, X

In [ ]:
def asianmc_vec(S0, K, T, r, sigma, q, N, numsim=10000, av=False):
    """Monte Carlo price of an arithmetic average Asian call. 
    Pass `av=True` to use antithetic sampling."""
    X0 = np.log(S0)
    nu = r-q-.5*sigma**2    
    _, X = bmsim_vec(T, N, X0, nu, sigma, numsim, av)
    S = np.exp(X)
    payoffs = np.maximum(S[:, 1:].mean(axis=1)-K, 0.)
    g = np.exp(-r*T)*payoffs
    if av:
        g = .5*(g[:numsim]+g[numsim:])
    C = g.mean(); s = g.std()
    zq = norm.ppf(0.975)
    Cl = C - zq/np.sqrt(numsim)*s
    Cu = C + zq/np.sqrt(numsim)*s
    return C, Cl, Cu

In [ ]:
S0 = 11; K = 10; T = 3/12.; r = 0.02; sigma = .3; q = 0.01; N = 10; numsim=10000
np.random.seed(0)
C, Cl, Cu = asianmc_vec(S0, K, T, r, sigma, q, N, numsim, av=False)
C, Cu-Cl

## CONTROL VARIATES 